# Spotify Song Suggester

## Team Members
- Sara West, DSPT8 Unit 4
- Yeraldina Estrella, DSPT9 Unit 3
- Eugene Sheely, DSPT9 Unit 3

## Project Description
A user will input some basic information about a song enjoyed. The song suggester will evaluate the song and find 5 similar songs the user may also enjoy.

In [1]:
# Import required packages
import numpy as np
import pandas as pd
import tensorflow as tf

print(f'Numpy Version: {np.__version__}')
print(f'Pandas Version: {np.__version__}')
print(f'Tensorflow Version: {tf.__version__}')

Numpy Version: 1.19.5
Pandas Version: 1.19.5
Tensorflow Version: 2.4.1


In [2]:
!pip install sklearn

In [3]:
!pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 3.4MB/s 


In [4]:
from sklearn.neighbors import NearestNeighbors
import category_encoders as ce 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

## Load Data

In [6]:
# from google.colab import drive, files
# uploaded = files.upload()
# data files too large to store on GitHub
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


###Primary Dataset, Songs

In [7]:
# Load dataset
path ='/content/drive/My Drive/Data/spotify_data.csv'
data = pd.read_csv(path)
print(data.shape)
data.sample(4)

(174389, 19)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
131545,0.848,['Banda Sinaloense El Recodo De Cruz Lizarraga'],0.269,173827,0.5920,0,6r52JkwN4Lx2u865o4PYje,0.95200,10,0.2280,-9.482,1,Por una Mujer Casada,19,1963-09-11,0.0463,187.902,0.870,1963
11134,0.389,['Leo Sayer'],0.597,253227,0.3420,0,2TcwEYyydQuEMJwdmSgVLD,0.00003,8,0.1140,-8.167,1,When I Need You - Remastered,54,1976,0.0299,109.609,0.110,1976
81437,0.784,['Sonny Rollins'],0.485,471173,0.0735,0,6W25U2EbDixQbLbqpjz0TT,0.85500,6,0.0951,-20.188,1,Solitude,17,1957,0.0709,75.939,0.332,1957
115765,0.630,['Dave Brubeck'],0.601,146707,0.4700,0,2RpXpY7tp5RjbKopUuwlix,0.00007,5,0.2810,-13.341,0,"Theme From ""Mr. Broadway""",26,1966,0.0291,110.735,0.880,1966


In [8]:
data.drop(columns=['release_date'], inplace=True)
print(data.shape)
data.sample(4)

(174389, 18)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
120073,0.6350,"['Aretha Franklin', 'Marvis Staples']",0.332,368333,0.8080,0,7xaEfxxPbV1GlxNpCoDA11,0.000000,5,0.777,-8.463,1,"Oh Happy Day - Live at New Bethel Baptist Church, Detroit, MI - July 1987",36,0.0915,174.035,0.610,1987
89467,0.7450,['Duke Ellington'],0.611,179333,0.0975,0,02E4skBHp6ZyoEDgXHcQgu,0.856000,0,0.101,-11.570,1,Satin Doll,43,0.0289,88.835,0.399,1997
92150,0.0207,"['Justin Bieber', 'Boyz II Men']",0.658,185560,0.6790,0,5Ea0FNlMDWkraQDlfOPrDv,0.000003,10,0.198,-8.606,0,Fa La La,57,0.0361,94.489,0.420,2011
18671,0.4500,['Andrew Peterson'],0.492,279373,0.3380,0,078X5eXgOPNjn08B9YbMH4,0.000181,2,0.120,-11.799,1,Isn't It Love - 2014 Version,35,0.0298,107.856,0.134,2014


In [9]:
#check for exact duplicate rows
data.duplicated().sum()

2159

In [10]:
data.drop_duplicates(inplace=True)
print(data.shape)
print(data.id.nunique()) #verify only unique song id's remain

(172230, 18)
172230


In [11]:
data.sample(4)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
51886,0.986,"['Wolfgang Amadeus Mozart', 'Capella Istropolitana', 'Barry Wordsworth']",0.254,465000,0.0111,0,0CZBabrzal9Op6gEUdZJqV,0.314000,3,0.122,-22.698,1,"Symphony No. 40 in G Minor, K. 550: II. Andante",45,0.0376,97.510,0.0399,1988
144138,0.958,"['Suraiya', 'Hameeda Bano']",0.574,178422,0.4430,0,76yqbMdWyZCFt2pmeDrgkt,0.000000,1,0.494,-11.911,1,Bachpan Gaya Jawani Aai,0,0.0607,110.690,0.6860,1946
81896,0.713,['Kenny Drew'],0.475,438067,0.7490,0,1osjVaShUB9PdfBMZgXdN8,0.000458,2,0.185,-7.782,0,Undercurrent - Rudy Van Gelder Edition/2007 Remaster,19,0.0677,76.281,0.6700,1960
94583,0.991,"['Danai Stratigopoulou', 'Panos Visvardis']",0.392,182173,0.3160,0,4K9P9H5l0xJbUV89GKtKPc,0.001610,2,0.378,-9.650,1,Dio Maura Matia,0,0.0388,112.605,0.4550,1930


In [12]:
# view one song with multiple artists
data.iloc[62729]

acousticness                                                                                   0.919
artists             ['Richard Adler', 'Jerry Ross', 'The Original Broadway Cast Of The Pajama Game']
danceability                                                                                   0.595
duration_ms                                                                                   106440
energy                                                                                         0.139
explicit                                                                                           0
id                                                                            6kfWm0913KSuaYy8uTmWIX
instrumentalness                                                                                   0
key                                                                                                0
liveness                                                                                   

In [13]:
data.artists.nunique()

36195

In [14]:
# reduce artists column to only the first artist listed
data['artists'] = data['artists'].apply(lambda x: x.split(',',1)[0])
data['artists'] = data['artists'].apply(lambda x: x.strip("[']"))
data.artists.nunique()

23169

In [15]:
# verify artist column is as expected
data.iloc[62729,1]

'Richard Adler'

In [16]:
data.dtypes

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

### Genres dataset

In [17]:
# genres per artist
path ='/content/drive/My Drive/Data/data_w_genres.csv'
data_g = pd.read_csv(path)
print(data_g.shape)
data_g.sample(4)

(32539, 16)


,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
24535,Ruined Conflict,0.000947,0.4050,264000.0,0.688,0.637,0.1880,-7.562,0.0359,140.027,0.0615,10.0,1,1,2,"['dark electro', 'futurepop']"
5883,Craig Jessop,0.969000,0.0966,287200.0,0.170,0.843,0.1540,-23.428,0.0384,89.328,0.1500,38.0,10,1,2,[]
13037,James Harcourt,0.016700,0.8050,419736.0,0.631,0.752,0.0377,-8.443,0.0995,126.026,0.5770,0.0,10,0,2,[]
5274,Christmas Cello Music Orchestra,0.534000,0.4420,180000.0,0.543,0.919,0.1700,-15.651,0.0396,108.041,0.1160,0.0,9,1,2,['christmas product']


In [18]:
# reduce data_g to only the pertinent columns
data_g = data_g[['artists', 'genres']]
data_g.sample(5)

,artists,genres
15664,Klaus Hermanns,[]
28783,The Original,"['broadway', 'show tunes']"
21073,OLIVER GIL,[]
16087,Lagwagon,"['melodic hardcore', 'punk', 'ska punk', 'skate punk', 'socal pop punk']"
19440,Mike Douglas,['movie tunes']


In [19]:
data_g.artists.nunique()

32539

In [20]:
data_g.isnull().sum()

artists    0
genres     0
dtype: int64

In [21]:
# create "not specified" genre for empty sets
data_g['genres'] = data_g['genres'].apply(lambda x: str(x).replace("[]","['not specified']"))
data_g.sample(5)

,artists,genres
3093,Big Pun,"['bronx hip hop', 'east coast hip hop', 'gangster rap', 'hardcore hip hop', 'hip hop', 'hip pop', 'rap', 'southern hip hop']"
17540,Luis Vargas,"['bachata', 'bachata dominicana', 'dominican pop', 'latin', 'tropical']"
13656,Jeziel Quintela,['not specified']
13609,Jesse Lee Brooks,['not specified']
16218,Laszlo Heltay,['not specified']


###New Dataset, Songs with Genres

In [22]:
df = pd.merge(data, data_g, how="left", on = ["artists", "artists"])
print(df.shape)
df.sample(4)

(172230, 19)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
154846,0.00599,Mr. Bizz,0.804,390326,0.229,0,5AvNe4swiuDfy3nhPQS0nq,0.926000,6,0.109,-16.510,1,Jantra,0,0.0937,124.998,0.295,2010,['ambient dub techno']
165489,0.13400,Tom Scott,0.551,305467,0.336,0,6kvdI7vd2LRBvu0qGHTJUi,0.012800,5,0.070,-18.199,1,Amaretto,24,0.0290,79.847,0.235,1987,"['jazz funk', 'smooth jazz']"
49005,0.00313,Led Zeppelin,0.334,503840,0.798,0,6Xfw7G8EZXh9QIh8faW6q4,0.023900,5,0.964,-8.289,0,Since I've Been Loving You - (Live at MSG 1973) [Remaster],42,0.0478,112.089,0.410,1976,"['album rock', 'classic rock', 'hard rock', 'rock']"
135025,0.86900,Los Alegres De Teran,0.489,188773,0.379,0,3AY2QF9wmdKgiCzVIQeMVX,0.000004,4,0.262,-5.143,1,Caminos De Guanajuato,33,0.1020,184.868,0.798,1987,['norteno']


In [23]:
df.isnull().sum() # It appears that not all artists in the song list were included in the genre list

acousticness           0
artists                0
danceability           0
duration_ms            0
energy                 0
explicit               0
id                     0
instrumentalness       0
key                    0
liveness               0
loudness               0
mode                   0
name                   0
popularity             0
speechiness            0
tempo                  0
valence                0
year                   0
genres              2174
dtype: int64

In [24]:
# fill in empty genres
df['genres'].fillna("['not specified']", inplace = True)
df.isnull().sum()

acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
speechiness         0
tempo               0
valence             0
year                0
genres              0
dtype: int64

## Data Exploration

In [25]:
# Descriptive statistics for key fields
df.describe()
# Add visualizations (where applicable)

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
count,172230.000000,172230.000000,1.722300e+05,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000
mean,0.501914,0.536246,2.326718e+05,0.480989,0.067956,0.194968,5.204395,0.211396,-11.777780,0.703147,25.925913,0.105896,116.937762,0.525249,1976.582001
std,0.379394,0.175858,1.471768e+05,0.272032,0.251670,0.332974,3.517851,0.180661,5.690105,0.456873,21.866745,0.183053,30.290836,0.264392,26.708673
min,0.000000,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1920.000000
25%,0.091300,0.414000,1.662000e+05,0.248000,0.000000,0.000000,2.000000,0.099300,-14.935750,0.000000,1.000000,0.035200,93.834000,0.311000,1955.000000
50%,0.523000,0.547000,2.057175e+05,0.463000,0.000000,0.000498,5.000000,0.138000,-10.867000,1.000000,26.000000,0.045400,115.657000,0.537000,1977.000000
75%,0.896000,0.668000,2.656000e+05,0.708000,0.000000,0.238000,8.000000,0.270000,-7.532000,1.000000,42.000000,0.076000,134.991000,0.744000,1998.000000
max,0.996000,0.988000,5.338302e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.971000,243.507000,1.000000,2021.000000


In [26]:
df.describe(include=[object])

,artists,id,name,genres
count,172230,172230,172230,172230
unique,23169,172230,137013,9512
top,Francisco Canaro,3C9oV51pSbrOJqcTrMdVyw,White Christmas,['not specified']
freq,2228,1,103,27923


In [27]:
print(round(100*27923/172230, 2), '% of songs have no genre specified')

16.21 % of songs have no genre specified


In [28]:
df['artists'] = df.artists.astype(str).apply(lambda x: x.lower())
df['name'] = df.name.astype(str).apply(lambda x: x.lower())
df['genres'] = df.genres.astype(str).apply(lambda x: x.lower())
df.sample()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
49777,0.829,nicola di bari,0.541,197733,0.415,0,7nxkNNVzWpuoBHSBoeKoYJ,0.00342,2,0.253,-13.981,1,el corazon es un gitano,51,0.0288,104.136,0.706,1980,"['classic italian pop', 'pop romantico']"


In [29]:
df.sample()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
120722,0.146,2pac,0.772,219720,0.936,1,65rPkyDC6FktCVkZIAtXaZ,0.00103,7,0.293,-4.136,1,nothing to lose,43,0.0678,103.443,0.656,1997,"['g funk', 'gangster rap', 'hip hop', 'rap', 'west coast rap']"


## Baseline Model (Benchmark)

In [30]:
# Encode catagorical features
encoder = ce.OrdinalEncoder(cols=['artists', 'name', 'genres', 'id'])
encoder.fit(df)
print('')

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [31]:
dg = encoder.transform(df)
dg.dtypes

acousticness        float64
artists               int64
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                    int64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                  int64
popularity            int64
speechiness         float64
tempo               float64
valence             float64
year                  int64
genres                int64
dtype: object

In [49]:
# test input data
test_data = {
  "artists": 'johnny cash',
  "name": "I Walk the Line"
  # "name": "i walk the line"
}
test_data['artists'] = test_data['artists'].lower()
test_data['name'] = test_data['name'].lower()

In [50]:
test_row=df[df['artists'].str.match(test_data['artists']) & df['name'].str.match(test_data['name'])]
test_row

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
7314,0.798,johnny cash,0.862,163046,0.484,0,1Vr6JSWTpl5MIxUHOJ3nny,0.000996,10,0.1050,-11.183,1,i walk the line,48,0.0421,105.834,0.787,1957,"['arkansas country', 'outlaw country']"
8683,0.970,johnny cash,0.596,154173,0.194,0,3X9tJw6z5LowFx339fhTvu,0.162000,10,0.0990,-20.389,1,i walk the line - stereo version,63,0.0347,114.974,0.746,1964,"['arkansas country', 'outlaw country']"
9885,0.877,johnny cash,0.713,176533,0.258,0,7hxZF4jETnE5Q75rKQnMjE,0.006060,9,0.0967,-14.590,1,i walk the line,67,0.0384,98.805,0.560,1970,"['arkansas country', 'outlaw country']"
26709,0.798,johnny cash,0.862,163046,0.484,0,5WwSSnn7t4Ktu5eOHzI39D,0.000996,10,0.1050,-11.183,1,i walk the line,26,0.0421,105.834,0.787,1958,"['arkansas country', 'outlaw country']"
27738,0.872,johnny cash,0.611,154547,0.381,0,0SHqu1Nlm2Cu6eNbmxHvRA,0.698000,10,0.1020,-15.462,1,i walk the line,41,0.0347,115.148,0.706,1964,"['arkansas country', 'outlaw country']"
63159,0.798,johnny cash,0.862,163046,0.484,0,4w9zNH0jhhHOc08YLAN9oe,0.000996,10,0.1050,-11.183,1,i walk the line,15,0.0421,105.834,0.787,1956,"['arkansas country', 'outlaw country']"
63372,0.646,johnny cash,0.862,162112,0.413,0,4AItZ329ewZOhqmxrSPzDd,0.001510,10,0.1020,-14.829,1,i walk the line (2017 remaster),18,0.0474,105.859,0.792,1957,"['arkansas country', 'outlaw country']"
63545,0.615,johnny cash,0.855,162629,0.417,0,1QOW1JEWkbYEOyBlq81P2j,0.001330,10,0.1010,-14.995,1,i walk the line (2017 remaster),17,0.0471,105.842,0.792,1958,"['arkansas country', 'outlaw country']"
99244,0.510,johnny cash,0.553,134573,0.661,0,2bItf9pag16H6HtjLOqzSS,0.000007,10,0.8860,-10.432,1,"i walk the line - live at san quentin state prison, san quentin, ca - february 1969",29,0.0380,120.813,0.772,1969,"['arkansas country', 'outlaw country']"
112791,0.543,johnny cash,0.820,158169,0.365,0,0SoX8PxWYqL4zpuzuai35A,0.000102,10,0.1020,-14.536,0,"i walk the line (alternate version, take 1) (2017 remaster)",12,0.0442,102.355,0.779,1957,"['arkansas country', 'outlaw country']"


In [51]:
test_e = encoder.transform(test_row)
test_e

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
7314,0.798,591,0.862,163046,0.484,0,7315,0.000996,10,0.1050,-11.183,1,6350,48,0.0421,105.834,0.787,1957,209
8683,0.970,591,0.596,154173,0.194,0,8684,0.162000,10,0.0990,-20.389,1,7409,63,0.0347,114.974,0.746,1964,209
9885,0.877,591,0.713,176533,0.258,0,9886,0.006060,9,0.0967,-14.590,1,6350,67,0.0384,98.805,0.560,1970,209
26709,0.798,591,0.862,163046,0.484,0,26710,0.000996,10,0.1050,-11.183,1,6350,26,0.0421,105.834,0.787,1958,209
27738,0.872,591,0.611,154547,0.381,0,27739,0.698000,10,0.1020,-15.462,1,6350,41,0.0347,115.148,0.706,1964,209
63159,0.798,591,0.862,163046,0.484,0,63160,0.000996,10,0.1050,-11.183,1,6350,15,0.0421,105.834,0.787,1956,209
63372,0.646,591,0.862,162112,0.413,0,63373,0.001510,10,0.1020,-14.829,1,53613,18,0.0474,105.859,0.792,1957,209
63545,0.615,591,0.855,162629,0.417,0,63546,0.001330,10,0.1010,-14.995,1,53613,17,0.0471,105.842,0.792,1958,209
99244,0.510,591,0.553,134573,0.661,0,99245,0.000007,10,0.8860,-10.432,1,81061,29,0.0380,120.813,0.772,1969,209
112791,0.543,591,0.820,158169,0.365,0,112792,0.000102,10,0.1020,-14.536,0,91436,12,0.0442,102.355,0.779,1957,209


In [52]:
# Simple baseline model
knn = NearestNeighbors(n_neighbors=6)
knn.fit(dg)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [53]:
results = knn.kneighbors(test_e)
results

(array([[   0.        ,  462.68739038,  467.43894614,  548.20663532,
          669.3421918 ,  710.6548821 ],
        [   0.        ,  858.69237713,  881.26876405,  888.50045337,
          991.38623158, 1009.04487256],
        [   0.        , 1461.13796917, 1745.7654159 , 1748.6873901 ,
         1856.23157158, 1882.86933181],
        [   0.        , 1088.49235634, 1224.0396489 , 1345.82303687,
         1355.02939839, 1511.05107103],
        [   0.        , 1034.13932999, 1318.77930036, 1955.96629328,
         2194.61604491, 2198.38282232],
        [   0.        ,  505.4382458 ,  831.84121352, 1308.0361335 ,
         1650.26163046, 2090.25345275],
        [   0.        ,  500.0662572 ,  545.17889621,  561.42006091,
          629.69499447,  795.45178155],
        [   0.        ,  493.50524315,  545.17889621,  646.40199836,
          809.14995039,  865.77842567],
        [   0.        ,  761.06342386,  823.7675503 ,  850.96815889,
         1095.53984385, 1130.49463289],
        [   0.     

In [54]:
j = [results[1][0]]
j

[array([7314, 7221, 7259, 7074, 6932, 6797])]

In [55]:
df.iloc[j[0]]

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
7314,0.798,johnny cash,0.862,163046,0.484,0,1Vr6JSWTpl5MIxUHOJ3nny,0.000996,10,0.1050,-11.183,1,i walk the line,48,0.0421,105.834,0.787,1957,"['arkansas country', 'outlaw country']"
7221,0.722,dizzy gillespie,0.713,162760,0.462,0,43luWh28le4rVtu7kylUnR,0.017300,5,0.1010,-13.108,0,tin tin deo,34,0.0312,137.407,0.967,1956,"['bebop', 'big band', 'contemporary post-bop', 'cool jazz', 'hard bop', 'jazz', 'jazz fusion', 'jazz trumpet', 'latin jazz', 'soul jazz', 'swing', 'vocal jazz']"
7259,0.675,ella fitzgerald,0.626,163333,0.100,0,5JfYrlqAHAY7ZYMoEua53B,0.000000,7,0.0852,-14.999,0,i've got you under my skin,34,0.0523,135.387,0.698,1956,"['adult standards', 'jazz blues', 'swing', 'vocal jazz']"
7074,0.676,frank sinatra,0.547,162840,0.208,0,3YCbx1QqGrnxMsPEZLC1CT,0.000000,9,0.1000,-13.458,1,love is here to stay - remastered 1998,58,0.0334,108.999,0.424,1956,"['adult standards', 'easy listening', 'lounge']"
6932,0.893,asha bhosle,0.538,163469,0.396,0,0BXh2Vi0il5tC5OHBFDMMM,0.302000,6,0.2760,-12.629,0,chanda mama door ke,25,0.0475,176.937,0.763,1955,"['chutney', 'classic bollywood', 'desi pop', 'filmi', 'sufi']"
6797,0.915,doris day,0.391,162813,0.101,0,51SBVSzsXvRaHzEHNE24bZ,0.000000,9,0.1620,-13.862,1,hold me in your arms (with percy faith & his orchestra) - 78rpm version,27,0.0415,81.209,0.148,1954,"['adult standards', 'brill building pop', 'easy listening', 'lounge', 'vocal jazz']"


## Modeling Approach #1

## Modeling Approach #2

## Hyper-parameter Tuning for Best Model

## Save Final Model

In [ ]:
import joblib

joblib.dump(model, '../models/model_name.joblib')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')